After creating the workspace, download the config.json file and save it in a folder relative to the path defined on the config_path to run

In [4]:
from azure.identity import AzureCliCredential
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
#import mlflow
import argparse
import pandas as pd
from azure.ai.ml import MLClient

credentials = AzureCliCredential(tenant_id='79fa5f3d-6f5b-4abe-a5bc-25abcb328320')

try:
    ml_client = MLClient.from_config(credential=credentials)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": "a8b0cced-98ab-4669-bf6a-a77f4c102009",
        "resource_group": "MLops_resource",
        "workspace_name": "MLops_workspaces",
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../config/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credentials, path=config_path)
print(ml_client)

from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_E16S_V3", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()
    

Found the config file in: ../config/config.json


MLClient(credential=<azure.identity._credentials.azure_cli.AzureCliCredential object at 0x7fcf0f459880>,
         subscription_id=a8b0cced-98ab-4669-bf6a-a77f4c102009,
         resource_group_name=MLops_resource,
         workspace_name=MLops_workspaces)


In [5]:
import os

# create a folder for the script files
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

src folder created


In [6]:
%%writefile $script_folder/prep.py
"""
Prepares raw data and provides training, validation and test datasets
"""

import argparse

from pathlib import Path
import os
import numpy as np
import pandas as pd

import mlflow

TARGET_COL = "cost"

NUMERIC_COLS = [
    "distance", "dropoff_latitude", "dropoff_longitude", "passengers", "pickup_latitude",
    "pickup_longitude", "pickup_weekday", "pickup_month", "pickup_monthday", "pickup_hour",
    "pickup_minute", "pickup_second", "dropoff_weekday", "dropoff_month", "dropoff_monthday",
    "dropoff_hour", "dropoff_minute", "dropoff_second"
]

CAT_NOM_COLS = [
    "store_forward", "vendor"
]

CAT_ORD_COLS = [
]

def parse_args():
    '''Parse input arguments'''

    parser = argparse.ArgumentParser("prep")
    parser.add_argument("--raw_data", type=str, help="Path to raw data")
    parser.add_argument("--train_data", type=str, help="Path to train dataset")
    parser.add_argument("--val_data", type=str, help="Path to test dataset")
    parser.add_argument("--test_data", type=str, help="Path to test dataset")
    
    #parser.add_argument("--enable_monitoring", type=str, help="enable logging to ADX")
    #parser.add_argument("--table_name", type=str, default="mlmonitoring", help="Table name in ADX for logging")
    
    args = parser.parse_args()

    return args

#def log_training_data(df, table_name):
#    from obs.collector import Online_Collector
#    collector = Online_Collector(table_name)
#    collector.batch_collect(df)

def main(args):
    '''Read, split, and save datasets'''

    # ------------ Reading Data ------------ #
    # -------------------------------------- #

    data = pd.read_csv((Path(args.raw_data)))
    data = data[NUMERIC_COLS + CAT_NOM_COLS + CAT_ORD_COLS + [TARGET_COL]]

    # ------------- Split Data ------------- #
    # -------------------------------------- #

    # Split data into train, val and test datasets

    random_data = np.random.rand(len(data))

    msk_train = random_data < 0.7
    msk_val = (random_data >= 0.7) & (random_data < 0.85)
    msk_test = random_data >= 0.85

    train = data[msk_train]
    val = data[msk_val]
    test = data[msk_test]

    mlflow.log_metric('train size', train.shape[0])
    mlflow.log_metric('val size', val.shape[0])
    mlflow.log_metric('test size', test.shape[0])

    train.to_csv((Path(args.train_data) / "train.csv"), index=False)
    val.to_csv((Path(args.val_data) / "val.csv"), index=False)
    test.to_csv((Path(args.test_data) / "test.csv"), index=False)

    #if (args.enable_monitoring.lower == 'true' or args.enable_monitoring == '1' or args.enable_monitoring.lower == 'yes'):
    #    log_training_data(data, args.table_name)


if __name__ == "__main__":

    mlflow.start_run()

    # ---------- Parse Arguments ----------- #
    # -------------------------------------- #

    args = parse_args()

    lines = [
        f"Raw data path: {args.raw_data}",
        f"Train dataset output path: {args.train_data}",
        f"Val dataset output path: {args.val_data}",
        f"Test dataset path: {args.test_data}",

    ]

    for line in lines:
        print(line)
    
    main(args)

    mlflow.end_run()

Writing src/prep.py


In [7]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

prep_data = command(
    inputs=dict(
        prep_input=Input(
            type="uri_file",
            path="/Users/ejenamvictor/Desktop/mlops_classicRegression/data/taxi-data.csv",
        )
    ),
    outputs=dict(
        train_data=Output(type="uri_folder",
                          path='azureml://subscriptions/a8b0cced-98ab-4669-bf6a-a77f4c102009/resourcegroups/MLops_practice/workspaces/mlops_workspaces/datastores/workspaceblobstore/paths/LocalUpload/9292ec840b5d1db6306dba71da69ab7f/train_data',
                          mode="rw_mount"),
        val_data=Output(type="uri_folder",
                        path='azureml://subscriptions/a8b0cced-98ab-4669-bf6a-a77f4c102009/resourcegroups/MLops_practice/workspaces/mlops_workspaces/datastores/workspaceblobstore/paths/LocalUpload/9292ec840b5d1db6306dba71da69ab7f/val_data',
                        mode="rw_mount"),
        test_data=Output(type="uri_folder",
                         path='azureml://subscriptions/a8b0cced-98ab-4669-bf6a-a77f4c102009/resourcegroups/MLops_practice/workspaces/mlops_workspaces/datastores/workspaceblobstore/paths/LocalUpload/9292ec840b5d1db6306dba71da69ab7f/test_data',
                         mode="rw_mount"),
    ),
    code="./src/",  # location of source code
    command="python prep.py --raw_data ${{inputs.prep_input}} --train_data ${{outputs.train_data}} --val_data ${{outputs.val_data}} --test_data ${{outputs.test_data}}",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="cpu-cluster", # delete this line to use serverless compute
    display_name="preprocess-autolog",
    experiment_name="pre_process"
)


In [8]:
returned_job = ml_client.jobs.create_or_update(prep_data)
returned_job

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.0 MBs): 100%|████████

Experiment,Name,Type,Status,Details Page
pre_process,jovial_wire_q2b28h8smn,command,Starting,Link to Azure Machine Learning studio


In [9]:
import os

train_src_dir = "./src/train"
os.makedirs(train_src_dir, exist_ok=True)

In [10]:
%%writefile {train_src_dir}/train.py

from matplotlib import pyplot as plt
import numpy as np
import argparse
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import os
import pandas as pd
import mlflow
import mlflow.sklearn

def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])


# Start Logging
mlflow.start_run()

# enable autologging
mlflow.sklearn.autolog()

os.makedirs("./outputs", exist_ok=True)

TARGET_COL = "cost"

NUMERIC_COLS = [
    "distance", "dropoff_latitude", "dropoff_longitude", "passengers", "pickup_latitude",
    "pickup_longitude", "pickup_weekday", "pickup_month", "pickup_monthday", "pickup_hour",
    "pickup_minute", "pickup_second", "dropoff_weekday", "dropoff_month", "dropoff_monthday",
    "dropoff_hour", "dropoff_minute", "dropoff_second"
]

CAT_NOM_COLS = [
    "store_forward", "vendor"
]

CAT_ORD_COLS = [
]
    

def main():
    """Main function of the script."""
    
    
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data', type=str)
    parser.add_argument("--n_estimators", dest='n_estimators', type=int, default=500)
    parser.add_argument("--max_depth", dest='max_depth', type=int, default=1)
    parser.add_argument('--max_features', type=str, default='auto',
                        help='Number of features to consider at every split')
    parser.add_argument("--min_samples_leaf", dest='min_samples_leaf', type=int, default=4)
    parser.add_argument("--min_samples_split", dest='min_samples_split', type=int, default=5)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    parser.add_argument("--model", type=str, help="path to model file")
    # parse args
    args = parser.parse_args()
    
    # paths are mounted as folder, therefore, we are selecting the file from folder
    train_data = pd.read_csv(select_first_file(args.training_data))

    
    # Split the data into input(X) and output(y)
    y_train = train_data[TARGET_COL]
    X_train = train_data[NUMERIC_COLS + CAT_NOM_COLS + CAT_ORD_COLS]
    
    
    # Train a Random Forest Regression Model with the training set
    model = RandomForestRegressor(n_estimators = args.n_estimators,
                                  max_depth = args.max_depth,
                                  max_features = args.max_features,
                                  min_samples_leaf = args.min_samples_leaf,
                                  min_samples_split = args.min_samples_split,
                                  random_state=0)
    
    
    # log model hyperparameters
    mlflow.log_param("model", "RandomForestRegressor")
    mlflow.log_param("n_estimators", args.n_estimators)
    mlflow.log_param("max_depth", args.max_depth)
    mlflow.log_param("max_features", args.max_features)
    mlflow.log_param("min_samples_leaf", args.min_samples_leaf)
    mlflow.log_param("min_samples_split", args.min_samples_split)
    

    # Train model with the train set
    model.fit(X_train, y_train)

    # Predict using the Regression Model
    yhat_train = model.predict(X_train)

    # Evaluate Regression performance with the train set
    r2 = r2_score(y_train, yhat_train)
    mse = mean_squared_error(y_train, yhat_train)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_train, yhat_train)
    
    # log model performance metrics
    mlflow.log_metric("train r2", r2)
    mlflow.log_metric("train mse", mse)
    mlflow.log_metric("train rmse", rmse)
    mlflow.log_metric("train mae", mae)

    # Visualize results
    plt.scatter(y_train, yhat_train,  color='black')
    plt.plot(y_train, y_train, color='blue', linewidth=3)
    plt.xlabel("Real value")
    plt.ylabel("Predicted value")
    plt.savefig("regression_results.png")
    mlflow.log_artifact("regression_results.png")
    
    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=model,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=model,
        path=os.path.join(args.model, "trained_model"),
    )
    
    # Stop Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()

Writing ./src/train/train.py


In [11]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

train_data = command(
    inputs={
        "train_input":Input(
            type="uri_folder",
            path="azureml://subscriptions/a8b0cced-98ab-4669-bf6a-a77f4c102009/resourcegroups/MLops_resource/workspaces/MLops_workspaces/datastores/workspaceblobstore/paths/LocalUpload/9292ec840b5d1db6306dba71da69ab7f/train_data/",
            #mode="rw_mount",
        ),
        "n_estimators": 100,
        "max_depth": 10,
        "max_features": "auto",
        "min_samples_leaf": 1,
        "min_samples_split": 2,
        "registered_model_name": "RFregressor"
    },
    outputs={
        "model_output":Output(type="uri_folder",
                          path='azureml://subscriptions/a8b0cced-98ab-4669-bf6a-a77f4c102009/resourcegroups/MLops_resource/workspaces/MLops_workspaces/datastores/workspaceblobstore/paths/LocalUpload/9292ec840b5d1db6306dba71da69ab7f/model_folder',)
                          },
    code="./src/train/",  # location of source code
    command="python train.py --training_data ${{inputs.train_input}} --n_estimators ${{inputs.n_estimators}} --max_depth ${{inputs.max_depth}} --max_features ${{inputs.max_features}} --min_samples_leaf ${{inputs.min_samples_leaf}}\
    --min_samples_split ${{inputs.min_samples_split}} --registered_model_name ${{inputs.registered_model_name}} --model ${{outputs.model_output}} ", 
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="cpu-cluster", # delete this line to use serverless compute
    display_name="traininig-autolog",
    experiment_name="taxi-training",
)

In [12]:
train_job = ml_client.jobs.create_or_update(train_data)
train_job

Experiment,Name,Type,Status,Details Page
taxi-training,loyal_rain_jh0bd5ppx6,command,Starting,Link to Azure Machine Learning studio


In [13]:
import os

train_src_dir = "./src/evaluate"
os.makedirs(train_src_dir, exist_ok=True)

In [14]:
%%writefile {train_src_dir}/evaluate.py
import argparse
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import os
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
from mlflow.tracking import MlflowClient


def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])



TARGET_COL = "cost"

NUMERIC_COLS = [
    "distance", "dropoff_latitude", "dropoff_longitude", "passengers", "pickup_latitude",
    "pickup_longitude", "pickup_weekday", "pickup_month", "pickup_monthday", "pickup_hour",
    "pickup_minute", "pickup_second", "dropoff_weekday", "dropoff_month", "dropoff_monthday",
    "dropoff_hour", "dropoff_minute", "dropoff_second"
]

CAT_NOM_COLS = [
    "store_forward", "vendor"
]

CAT_ORD_COLS = [
]
# Start Logging
##### mlflow.start_run()

# enable autologging
mlflow.sklearn.autolog()

os.makedirs("./outputs", exist_ok=True)

def parse_args():
    '''Parse input arguments'''

    parser = argparse.ArgumentParser("predict")
    parser.add_argument("--model_name", type=str, help="Name of registered model")
    parser.add_argument("--model_input", type=str, help="Path of input model")
    parser.add_argument("--test_data", type=str, help="Path to test dataset")
    parser.add_argument("--evaluation_output", type=str, help="Path of eval results")

    args = parser.parse_args()

    return args

def main(args):
    '''Read trained model and test dataset, evaluate model and save result'''

    # Load the test data
    test_data = pd.read_csv(select_first_file(args.test_data))

    # Split the data into inputs and outputs
    y_test = test_data[TARGET_COL]
    X_test = test_data[NUMERIC_COLS + CAT_NOM_COLS + CAT_ORD_COLS]

    # Load the model from input port
    model =  mlflow.sklearn.load_model(args.model_input) 

    # ---------------- Model Evaluation ---------------- #
    yhat_test, score = model_evaluation(X_test, y_test, model, args.evaluation_output)

    # ----------------- Model Promotion ---------------- #
    predictions, deploy_flag = model_promotion(args.model_name, args.evaluation_output, X_test, y_test, yhat_test, score)



def model_evaluation(X_test, y_test, model, evaluation_output):

    # Get predictions to y_test (y_test)
    yhat_test = model.predict(X_test)

    # Save the output data with feature columns, predicted cost, and actual cost in csv file
    output_data = X_test.copy()
    output_data["real_label"] = y_test
    output_data["predicted_label"] = yhat_test
    output_data.to_csv((Path(evaluation_output) / "predictions.csv"))

    # Evaluate Model performance with the test set
    r2 = r2_score(y_test, yhat_test)
    mse = mean_squared_error(y_test, yhat_test)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, yhat_test)

    # Print score report to a text file
    (Path(evaluation_output) / "score.txt").write_text(
        f"Scored with the following model:\n{format(model)}"
    )
    with open((Path(evaluation_output) / "score.txt"), "a") as outfile:
        outfile.write("Mean squared error: {mse.2f} \n")
        outfile.write("Root mean squared error: {rmse.2f} \n")
        outfile.write("Mean absolute error: {mae.2f} \n")
        outfile.write("Coefficient of determination: {r2.2f} \n")

    mlflow.log_metric("test r2", r2)
    mlflow.log_metric("test mse", mse)
    mlflow.log_metric("test rmse", rmse)
    mlflow.log_metric("test mae", mae)

    # Visualize results
    plt.scatter(y_test, yhat_test,  color='black')
    plt.plot(y_test, y_test, color='blue', linewidth=3)
    plt.xlabel("Real value")
    plt.ylabel("Predicted value")
    plt.title("Comparing Model Predictions to Real values - Test Data")
    plt.savefig("predictions.png")
    mlflow.log_artifact("predictions.png")

    return yhat_test, r2

def model_promotion(model_name, evaluation_output, X_test, y_test, yhat_test, score):
    
    scores = {}
    predictions = {}

    client = MlflowClient()

    for model_run in client.search_model_versions(f"name='{model_name}'"):
        model_version = model_run.version
        mdl = mlflow.pyfunc.load_model(
            model_uri=f"models:/{model_name}/{model_version}")
        predictions[f"{model_name}:{model_version}"] = mdl.predict(X_test)
        scores[f"{model_name}:{model_version}"] = r2_score(
            y_test, predictions[f"{model_name}:{model_version}"])

    if scores:
        if score >= max(list(scores.values())):
            deploy_flag = 1
        else:
            deploy_flag = 0
    else:
        deploy_flag = 1
    print(f"Deploy flag: {deploy_flag}")

    with open((Path(evaluation_output) / "deploy_flag"), 'w') as outfile:
        outfile.write(f"{int(deploy_flag)}")

    # add current model score and predictions
    scores["current model"] = score
    predictions["currrent model"] = yhat_test

    perf_comparison_plot = pd.DataFrame(
        scores, index=["r2 score"]).plot(kind='bar', figsize=(15, 10))
    perf_comparison_plot.figure.savefig("perf_comparison.png")
    perf_comparison_plot.figure.savefig(Path(evaluation_output) / "perf_comparison.png")

    mlflow.log_metric("deploy flag", bool(deploy_flag))
    mlflow.log_artifact("perf_comparison.png")

    return predictions, deploy_flag

if __name__ == "__main__":

    mlflow.start_run()

    args = parse_args()

    lines = [
        f"Model name: {args.model_name}",
        f"Model path: {args.model_input}",
        f"Test data path: {args.test_data}",
        f"Evaluation output path: {args.evaluation_output}",
    ]

    for line in lines:
        print(line)
    
    main(args)

    mlflow.end_run()

Writing ./src/evaluate/evaluate.py


In [15]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

evaluate_model = command(
    inputs={
        "model_input":Input(
            type="uri_file",
            path="azureml://subscriptions/a8b0cced-98ab-4669-bf6a-a77f4c102009/resourcegroups/MLops_resource/workspaces/MLops_workspaces/datastores/workspaceblobstore/paths/LocalUpload/9292ec840b5d1db6306dba71da69ab7f/model_folder/trained_model",
            #mode="rw_mount",
        ),
        "test_data":Input(type="uri_folder",
                         path="azureml://subscriptions/a8b0cced-98ab-4669-bf6a-a77f4c102009/resourcegroups/MLops_resource/workspaces/MLops_workspaces/datastores/workspaceblobstore/paths/LocalUpload/9292ec840b5d1db6306dba71da69ab7f/test_data/"),
        "model_name": "RFregressor",
    },
    outputs={
        "evaluation_output":Output(type="uri_folder",
                          path='azureml://subscriptions/a8b0cced-98ab-4669-bf6a-a77f4c102009/resourcegroups/MLops_resource/workspaces/MLops_workspaces/datastores/workspaceblobstore/paths/LocalUpload/9292ec840b5d1db6306dba71da69ab7f/eval_outputs',)
                          },
    code="./src/evaluate/",  # location of source code
    command="python evaluate.py --model_name ${{inputs.model_name}} --model_input ${{inputs.model_input}} --test_data ${{inputs.test_data}} --evaluation_output ${{outputs.evaluation_output}}", 
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="cpu-cluster", # delete this line to use serverless compute
    display_name="evalute_autolog",
    experiment_name="evaluation",
)

In [16]:
eval_job = ml_client.jobs.create_or_update(evaluate_model)
eval_job

Experiment,Name,Type,Status,Details Page
evaluation,green_yogurt_dmpjbc413y,command,Starting,Link to Azure Machine Learning studio


In [268]:
import uuid

registered_model_name = "RFregressor"

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

print(latest_model_version)



# Create a unique name for the endpoint
Taxiprice = "Taxi-price-endpoint-" + str(uuid.uuid4())[:8]

1


In [19]:
import os

train_src_dir = "./src/deploy"
os.makedirs(train_src_dir, exist_ok=True)

In [21]:
#%%writefile {train_src_dir}/deploy.py
#this should be run as a notebook
import uuid
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# Create a unique name for the endpoint
Taxiprice = "Taxi-price-endpoint-" + str(uuid.uuid4())[:8]

# Expect the endpoint creation to take a few minutes

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=Taxiprice,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.RandomForestRegressor",
    },
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

endpoint = ml_client.online_endpoints.get(name=Taxiprice)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name="RFregressor")]
)
print(f'Latest model is version "{latest_model_version}" ')

# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# Expect this deployment to take approximately 6 to 8 minutes.
# create an online deployment.
# if you run into an out of quota error, change the instance_type to a comparable VM that is available.\
# Learn more on https://azure.microsoft.com/en-us/pricing/details/machine-learning/.

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=Taxiprice,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment).result()

Overwriting ./src/deploy/deploy.py


In [252]:
#ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Updating local endpoint (taxi-price-endpoint-ba7fe200) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': 'enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://taxi-price-endpoint-ba7fe200.uksouth.inference.ml.azure.com/score', 'openapi_uri': 'https://taxi-price-endpoint-ba7fe200.uksouth.inference.ml.azure.com/swagger.json', 'name': 'taxi-price-endpoint-ba7fe200', 'description': 'this is an online endpoint for taxi price prediction', 'tags': {'train_data': 'taxi_price'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/a8b0cced-98ab-4669-bf6a-a77f4c102009/resourcegroups/mlops_resource/providers/microsoft.machinelearningservices/workspaces/mlops_workspaces/onlineendpoints/taxi-price-endpoint-ba7fe200', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/a8b0cced-98ab-4669-bf6a-a77f4c102009/providers/Microsoft.MachineLearningServices/locations/uksouth/mfeOperationsStatus/oe:e82f3418-797a-49f9-91f3-a65d4b1009f3:71127d78-29c4-4271-a1d2-18dd6fc6b3d2?api-version=2022-02-01-preview'}, 'print_as_yaml': True